<a href="https://colab.research.google.com/github/somkiatth/git/blob/master/ku_mllib_assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
telcodf = spark.read.csv('/content/Telco-Customer-Churn.csv',header=True, inferSchema=True)

In [ ]:
telcodf.show(5)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

In [ ]:
telcodf.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



In [ ]:
ttrain,ttest = telcodf.randomSplit([0.7,0.3])
ttrain.count(),ttest.count()

(4910, 2133)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
ttrain.write.csv('/content/ttrain')

AnalysisException: ignored

In [ ]:
df_loaded = spark.read.load('/content/ttrain',
                            format='csv')

In [ ]:
df_loaded.count()

5627

In [ ]:
df_loaded.show(5)

+----------+------+---+---+---+---+---+---+-----------+---+----+----+----+----+----+--------------+----+----------------+----+-------+----+
|       _c0|   _c1|_c2|_c3|_c4|_c5|_c6|_c7|        _c8|_c9|_c10|_c11|_c12|_c13|_c14|          _c15|_c16|            _c17|_c18|   _c19|_c20|
+----------+------+---+---+---+---+---+---+-----------+---+----+----+----+----+----+--------------+----+----------------+----+-------+----+
|0002-ORFBO|Female|  0|Yes|Yes|  9|Yes| No|        DSL| No| Yes|  No| Yes| Yes|  No|      One year| Yes|    Mailed check|65.6|  593.3|  No|
|0003-MKNFE|  Male|  0| No| No|  9|Yes|Yes|        DSL| No|  No|  No|  No|  No| Yes|Month-to-month|  No|    Mailed check|59.9|  542.4|  No|
|0004-TLHLJ|  Male|  0| No| No|  4|Yes| No|Fiber optic| No|  No| Yes|  No|  No|  No|Month-to-month| Yes|Electronic check|73.9| 280.85| Yes|
|0011-IGKFF|  Male|  1|Yes| No| 13|Yes| No|Fiber optic| No| Yes| Yes|  No| Yes| Yes|Month-to-month| Yes|Electronic check|98.0|1237.85| Yes|
|0013-EXCHZ|Female| 

solution 1)******

In [ ]:
ttrain.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [ ]:
ttrain.filter(ttrain.Churn.isNull()).collect()

[]

In [145]:
from pyspark.ml.feature import VectorAssembler

In [146]:
vecasm = VectorAssembler(
    inputCols=[ 'SeniorCitizen','tenure','MonthlyCharges', 'TotalCharges'],
    outputCol='features'
)

solution 2)*****

In [147]:
vectraindf=vecasm.transform(ttrain)

In [148]:
vectraindf.select(  'SeniorCitizen','tenure','MonthlyCharges', 'TotalCharges','features').show(truncate=False)

+-------------+------+--------------+------------+------------------------+
|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|features                |
+-------------+------+--------------+------------+------------------------+
|0            |9     |65.6          |593.3       |[0.0,9.0,65.6,593.3]    |
|0            |4     |73.9          |280.85      |[0.0,4.0,73.9,280.85]   |
|1            |13    |98.0          |1237.85     |[1.0,13.0,98.0,1237.85] |
|1            |3     |83.9          |267.4       |[1.0,3.0,83.9,267.4]    |
|0            |9     |69.4          |571.45      |[0.0,9.0,69.4,571.45]   |
|1            |71    |109.7         |7904.25     |[1.0,71.0,109.7,7904.25]|
|0            |63    |84.65         |5377.8      |[0.0,63.0,84.65,5377.8] |
|1            |7     |48.2          |340.35      |[1.0,7.0,48.2,340.35]   |
|0            |65    |90.45         |5957.9      |[0.0,65.0,90.45,5957.9] |
|0            |72    |116.8         |8456.75     |[0.0,72.0,116.8,8456.75]|
|0          

In [149]:
from pyspark.ml.feature import StringIndexer

In [150]:
si = StringIndexer(inputCol='Churn', outputCol='label_idx')

In [151]:
label_indexer = si.fit(vectraindf)

In [152]:
vec_idx_train_df = label_indexer.transform(vectraindf)

In [165]:
vec_idx_train_df.sample(1/10).show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+--------------------+---------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|            features|label_idx|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+

In [154]:
vec_idx_train_df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- label_idx: double (nullable = false)


solution 3)******

In [155]:
from pyspark.ml.classification import LogisticRegression

In [166]:
lr = LogisticRegression(featuresCol='features', labelCol='label_idx', maxIter=10, tol= .1, regParam=0)

In [167]:
lr_model =lr.fit(vec_idx_train_df)

Py4JJavaError: ignored

solution 4)*****

In [158]:
vec_idx_test_df =label_indexer.transform(vecasm.transform(ttest))

In [159]:
vec_idx_test_df.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+--------------------+---------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|            features|label_idx|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+

solution 5) ******

In [168]:
from pyspark.ml.feature import StandardScaler

In [169]:
std_scaler = StandardScaler(inputCol='features',outputCol='scaled_features')

In [172]:
std_scaler_model = std_scaler.fit(vec_idx_train_df)

Py4JJavaError: ignored